See the bottom two cells for the calculation of $Q_{\delta=5/3}$ and $Q_\mathrm{Rk}$ for the Solar Neighbourhood

In [2]:
from __future__ import annotations
import math
import numpy as np
from scipy.optimize import minimize
import numpy.typing as npt
import scipy

In [3]:
# Units adapted from:
"""
 * \file units.hpp
 * \brief Contains namespace Units
 * Gives various units in terms of code units (which are kpc,Myr,Msun)
 * 
 * C++ code written by Walter Dehnen, 1995/96
 * 
 * ********************************************************************************

  The unit system employed has:                                                

  angle:      measured in radian,                                              
  length:     measured in kilo parsec,                                         
  time:       measured in Mega years,                                          
  mass:       measured in solar masses.                                        

  This implies the following dimensions:                                        

  quantity        dimension / size                using other units         
  angular vel.  1 Myr^-1	                        = 977.775320024919 km/s/kpc  
  velocity      1 kpc/Myr                         = 977.775320024919 km/s   
  action/mass   1 kpc^2/Myr                       = 977.775320024919 kpc*km/s  
  potential     1 kpc^2/Myr^2                     = 956044.576449833 (km/s)^2  
  acceleration  1 kpc/Myr^2                                                 
  (or force per unit mass)
  G             4.49865897 E-12 kpc^3/Myr^2/Msun                 
                4.49865897 E-6 kpc^3/Gyr^2/Msun                 
  4 Pi G        5.65318158 E-11 kpc^3/Myr^2/Msun                               

  Note that this implies handy numbers for typical quantities in the Galaxy:   

  velocities 		are of the order of 0.1 to 0.4                         
  radii      		are of the order of 0.1 to 100                         
  dynamical times	are of the order of 1   to 1000                        
  G times total mass    is  of the order of 1            

 So, for example if you want to input a value of 200 km/s, you write it
 as 200. * Units::kms. If you want to know what a given velocity is in
 km/s you can output it as velocity_name * Units::kms_i (or,
 equivalently, velocity_name/Units::kms).
 """


# names of basic units
angle_unit    	= "radian"
length_unit   	= "kpc"
time_unit     	= "Myr"
mass_unit     	= "M_sun"
# basic units
radian = 1.
kpc = 1.
Myr = 1.
Msun = 1.
# angle
rad = radian                #!<  radian 
degree = rad * math.pi / 360. #!<  degrees = 360=circle 
arcmin = degree / 60.       #!<  arc minutes 
arcsec = arcmin / 60.       #!<  arc seconds 
mas = 0.001 * arcsec        #!<  milli arc seconds 
anghr = rad * math.pi / 24.   #!<  angle hour(24=circle 
angmin = anghr / 60.        #!<  angle minutes 
angsec = angmin / 60.       #!<  angle seconds 
# length
cm = kpc * 3.240778828894144e-22 #!<  centimeter  
meter = 1.e2 * cm                #!<  meter  
km = 1.e5 * cm                   #!<  kilo meter  
ly = 3.0660669364447e-4 * kpc    #!<  light year 
pc = 1.e-3 * kpc                 #!<  parsec  
Mpc = 1.e3 * kpc                 #!<  Mega parsec  
AU = arcsec * pc                 #!<  astronomical unit  
# time
sec = 3.168753556551954e-14 * Myr #!<  time second  
hour = 3600 * sec                 #!<  time hour 
day = 24 * sec                    #!<  time day 
yr = 1.e-6 * Myr                  #!<  year 
hyr = 1.e-4 * Myr                 #!<  hundred years 
century = hyr                     #!<  century 
Gyr = 1.e3 * Myr                  #!<  giga year 
Gyr2 = Gyr * Gyr                  #!<  giga year square
#  mass
gram = Msun / 1.989e33 #!<  gram 
kg = 1.e3 * gram       #!<  kilogram  
#  velocity
kpcMyr = kpc / Myr #!<  kpc per Myr 
kpcGyr = kpc / Gyr #!<  kpc per Gyr 
AUyr = AU / yr     #!<  AU per yr 
kms = km / sec     #!<  km per second 
c_light = ly / yr  #!<  speed of light 
#  acceleration
kpcMyr2 = kpc / (Myr * Myr) #!<  kpc per Myr 
kpcGyr2 = kpc / (Gyr * Gyr) #!<  kpc per Gyr 
kms2kpc = kms * kms / kpc   #!<  kpc per Gyr 
# angle velocity
radMyr = radian / Myr #!<  radian per Myr 
kmskpc = kms / kpc    #!<  kms per kpc 
masyr = mas / yr      #!<  milli arcsec per year 
ashyr = arcsec / hyr  #!<  arcsec per century 
secyr = angsec / yr   #!<  angsec per yr 
asyr = arcsec / yr    #!<  arcsec per yr 
# action
kpckms = kpc * kms #!<  kpc km per sec 
kpc2Gyr = kpc * kpcGyr #!<  kpc Kpc per Gyr 
# torque
kpc2Gyr2 = kpcGyr * kpcGyr #!<  kpc^2 per Gyr^2 
# energy
kmskms = kms * kms #!<  (km/s)^2 
# area
pc2 = pc * pc    #!<  square parsec 
kpc2 = kpc * kpc #!<  square kilo parsec 
cm2 = cm * cm    #!<  square centimeter 
#  volume
pc3 = pc2 * pc    #!<  cubic parsec 
kpc3 = kpc2 * kpc #!<  cubic kilo parsec 
cm3 = cm2 * cm    #!<  cubic centimeter 
# constant of gravity
G = 4.498658966346282e-12    #!<  Newtons G 
Grav = G                     #!<  Newtons G 
fPiG = 5.653181583871732e-11 #!<  4 Pi G 

# inverse quantities = useful for transformations
# inverse angle
rad_i = 1. / rad       #!<  inverse  
radian_i = 1. / radian #!<  inverse  
degree_i = 1. / degree #!<  inverse  
arcmin_i = 1. / arcmin #!<  inverse  
arcsec_i = 1. / arcsec #!<  inverse  
mas_i = 1. / mas       #!<  inverse  
anghr_i = 1. / anghr   #!<  inverse  
angmin_i = 1. / angmin #!<  inverse  
angsec_i = 1. / angsec #!<  inverse  
# inverse length
cm_i = 1. / cm       #!<  inverse  
meter_i = 1. / meter #!<  inverse  
km_i = 1. / km       #!<  inverse  
AU_i = 1. / AU       #!<  inverse  
ly_i = 1. / ly       #!<  inverse  
pc_i = 1. / pc       #!<  inverse  
kpc_i = 1. / kpc     #!<  inverse  
Mpc_i = 1. / Mpc     #!<  inverse  
# inverse time
sec_i = 1. / sec         #!<  inverse  
hour_i = 1. / hour       #!<  inverse  
day_i = 1. / day         #!<  inverse  
yr_i = 1. / yr           #!<  inverse  
hyr_i = 1. / hyr         #!<  inverse  
century_i = 1. / century #!<  inverse  
Myr_i = 1. / Myr         #!<  inverse  
Gyr_i = 1. / Gyr         #!<  inverse  
# inverse mass
gram_i = 1. / gram #!<  inverse  
kg_i = 1. / kg     #!<  inverse  
Msun_i = 1. / Msun #!<  inverse  
# inverse velocity
kpcMyr_i = 1. / kpcMyr   #!<  inverse  
kpcGyr_i = 1. / kpcGyr   #!<  inverse  
AUyr_i = 1. / AUyr       #!<  inverse  
kms_i = 1. / kms         #!<  inverse  
c_light_i = 1. / c_light #!<  inverse  
# inverse acceleration
kpcMyr2_i = 1. / kpcMyr2 #!<  inverse  
kpcGyr2_i = 1. / kpcGyr2 #!<  inverse  
kms2kpc_i = 1. / kms2kpc #!<  inverse  
# inverse angle velocity
radMyr_i = 1. / radMyr #!<  inverse  
kmskpc_i = 1. / kmskpc #!<  inverse  
masyr_i = 1. / masyr   #!<  inverse  
ashyr_i = 1. / ashyr   #!<  inverse  
secyr_i = 1. / secyr   #!<  inverse  
asyr_i = 1. / asyr     #!<  inverse  
# inverse action
kpckms_i = 1. / kpckms #!<  inverse  
kpc2Gyr_i = 1. / kpc2Gyr #!<  inverse  
# inverse torque
kpc2Gyr2_i = 1. / kpc2Gyr2 #!<  inverse  
# inverse energy
kmskms_i = 1. / kmskms #!<  inverse  
# inverse area
pc2_i = 1. / pc2   #!<  inverse  
kpc2_i = 1. / kpc2 #!<  inverse  
cm2_i = 1. / cm2   #!<  inverse  
# inverse volume
pc3_i = 1. / pc3   #!<  inverse  
kpc3_i = 1. / kpc3 #!<  inverse  
cm3_i = 1. / cm3   #!<  inverse  
# inverse of constant of gravity
G_i = 1. / G       #!<  inverse  
Grav_i = G_i       #!<  inverse  
fPiG_i = 1. / fPiG #!<  inverse  

In [42]:
def reduction_factor(s: float, chi: float | npt.NDArray[np.double], 
                     N_sum: int) -> float:
    if(isinstance(chi, np.ndarray)):
        assert(chi.shape == (1,))
        chi = chi[0]
    s_squared = s**2
    assert(s_squared <= 1)
    if(s_squared == 1):
        return 2*math.exp(-chi)/chi*scipy.special.iv(1, chi)
    else:
        output = 0.0
        for i in range(N_sum):
            output += (2*(1 - s_squared)*math.exp(-chi)/chi
                       *scipy.special.iv(i + 1, chi)
                       /(1 - s_squared/(i + 1)**2))
        return output

In [5]:
def F(kappa: float, components: tuple[npt.NDArray[np.double],
                                      npt.NDArray[np.double]], 
      N_rf: int, k: float) -> float:
    for i in range(2):
        assert(components[i].shape[1] == 2 or components[i].shape[0] == 0)
    n_g = components[0].shape[0]
    n_s = components[1].shape[0]
    kappa_squared = kappa**2
    k_squared = k**2
    chi_prefactor = k_squared/kappa_squared
    output = 0
    for i in range(n_g):
        output += components[0][i, 0]/(kappa_squared 
                                       + k_squared*components[0][i, 1]**2)
    for i in range(n_s):
        output += (components[1][i, 0]/kappa_squared
                   *reduction_factor(0, chi_prefactor*components[1][i, 1]**2, 
                                     N_rf))
    output *= 2*math.pi*G*k
    return output

In [6]:
def inverseF(k: float, *args) -> float:
    assert(len(args) == 3)
    return 1/F(args[0], args[1], args[2], k)

In [51]:
def Q_Rk(kappa: float, components: tuple[npt.NDArray[np.double],
                                         npt.NDArray[np.double]],
         k_max_factor: float = 10.0, 
         N_bessel_sum: int = 10) -> float:
    """Calculates $Q_\mathrm{Rk} (See Rafikov 2001, Romeo 2013)
    
    Arguments:
    kappa -- Local epicycle frequency
    components -- Length-2 tuple containing sets of disc components
                  consisting of gas and stars respectively. Each is
                  a 2D numpy array of shape (N_components, 2), where 
                  each "row" is the surface density and sound speed/
                  velocity dispersion (for gas/stars respectively)
                  of one component.

    Keyword arguments:
    k_max_factor -- Maximum $k$ value over which to minimise, divided
                    by $k_\mathrm{crit}$
    N_bessel_sum -- Number of terms included in the summation of the
                    stellar reduction factor (written as a weighted
                    sum of Bessel functions)
    """
    for i in range(2):
        assert(components[i].shape[1] == 2 or components[i].shape[0] == 0)

    total_surface_density = 0
    for i in range(2):
        for component in components[i]:
            total_surface_density += component[0]
    
    # Set bounds within which to minimise $k$
    k_crit = kappa**2/(2*math.pi*G*total_surface_density)
    k_bounds = [(0.0, k_max_factor*k_crit)]

    output = minimize(inverseF, k_crit, args=(kappa, components, 
                                              N_bessel_sum), 
                      bounds=k_bounds)
    return output.fun

<>:5: SyntaxWarning: invalid escape sequence '\m'
<>:5: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_211389/2018908392.py:5: SyntaxWarning: invalid escape sequence '\m'
  """Calculates $Q_\mathrm{Rk} (See Rafikov 2001, Romeo 2013)


In [54]:

def iterate_Q_delta(delta: float, kappa: float, 
                    components: tuple[npt.NDArray[np.double],
                                      npt.NDArray[np.double]],
                    Q_delta: float,
                    k_max_factor: float = 10.0, 
                    N_bessel_sum: int = 10) -> float:
    """Iterative function for calculating $Q_\delta$
    
    Arguments:
    delta -- Chosen $\delta$ value for $Q_\delta$. Our best
             value is $Q_{\delta=5/3}$. For $Q_\mathrm{Rk}$ 
             (see Rafikov 2001, Romeo 2013) set delta = 1
    kappa -- Local epicycle frequency
    components -- Length-2 tuple containing sets of disc components
                  consisting of gas and stars respectively. Each is
                  a 2D numpy array of shape (N_components, 2), where 
                  each "row" is the surface density and sound speed/
                  velocity dispersion (for gas/stars respectively)
                  of one component.
    Q_delta -- Initial value of Q_delta to be iterated on. Iterative 
               use of this function should converge if you start with
               1.
               
    Keyword arguments:
    k_max_factor -- Maximum $k$ value over which to minimise, divided
                    by $k_\mathrm{crit}$
    N_bessel_sum -- Number of terms included in the summation of the
                    stellar reduction factor (written as a weighted
                    sum of Bessel functions)
    """
    for i in range(2):
        assert(components[i].shape[1] == 2 or components[i].shape[0] == 0)
    
    # $Q_\delta$ is calculated in the same way as $Q_\mathrm{Rk}$ 
    # after we modify the sound speeds of the gas components to 
    # account for the different scaling of instability for gas 
    # vs stars.
    modified_components = (np.copy(components[0]),
                          np.copy(components[1]))
    for i in range(modified_components[0].shape[0]):
        modified_components[0][i, 1] *= Q_delta**(1 - 1/delta)
    
    return Q_Rk(kappa, modified_components, k_max_factor, N_bessel_sum)

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_211389/4015998088.py:7: SyntaxWarning: invalid escape sequence '\d'
  """Iterative function for calculating $Q_\delta$


In [58]:
# Epicycle frequency (Bland-Hawthorn & Gerhard 2016)
kappa = 41*kms/kpc

# Set age-velocity dispersion relation (Aumer & Binney 2009)
beta = 0.307
tau_1 = 0.001*Gyr
v_10 = 41.899*kms
avr = lambda age: v_10*((age + tau_1)/(10*Gyr + tau_1))**beta

# Set star formation rate history (Aumer & Binney 2009)
gamma = 0.117/Gyr
max_age = 12.5*Gyr
total_stellar_surface_density = 38*Msun/pc2
sfr_prefactor = (total_stellar_surface_density
                 /((math.exp(gamma*max_age) - 1)/gamma))
sfr = lambda age: sfr_prefactor*math.exp(gamma*age)

# Set stellar components
N_stellar = 100
delta_age = max_age/N_stellar
stellar_components = np.array([[delta_age*sfr((i + 0.5)/N_stellar*max_age),
                                avr((i + 0.5)/N_stellar*max_age)]
                               for i in range(N_stellar)])

# Set fluid components, combined tuple of components
# (Holmberg & Flynn 2000)
components = (np.array([[3.0*Msun/pc2, 4.0*kms],
                        [4.0*Msun/pc2, 7.0*kms],
                        [4.0*Msun/pc2, 9.0*kms],
                        [2.0*Msun/pc2, 40.0*kms]]),
              stellar_components)

# Multiply all gas component surface density by some factor to 
# calculate uncertainty in $Q$ due to the +/-50% uncertainty in 
# gas surface density (Holmberg & Flynn 2000)
error_factor = 0.5
for i in range(components[0].shape[0]):
    components[0][i, 0] *= error_factor

In [61]:
delta = 5/3

k_max_factor = 10
N_bessel_sum = 10

N_iterate = 10
Q_delta = 1

print("Q_Rk = ", Q_Rk(kappa, components, k_max_factor, N_bessel_sum))

print("\nQ_delta iterations:")
for i in range(N_iterate):
    Q_delta = iterate_Q_delta(delta, kappa, components, Q_delta, 
                              k_max_factor, N_bessel_sum)
    print(Q_delta)
print("\nQ_delta = ", Q_delta)

Q_Rk =  2.095111145584405

Q_delta iterations:
2.095111145584405
2.1322476584360412
2.1332071513085116
2.133231772930074
2.1332324046359714
2.1332324208424103
2.133232421258495
2.1332324212701397
2.1332324212691804
2.133232421269156

Q_delta =  2.133232421269156
